# El precio es correcto

Hoy construimos una solución más compleja para estimar los precios de los bienes.

1. Este cuaderno: crea una base de datos RAG con nuestros 400 000 datos de entrenamiento
2. Cuaderno del día 2.1: visualiza en 2D
3. Cuaderno del día 2.2: visualiza en 3D
4. Cuaderno del día 2.3: crea y prueba una canalización RAG con GPT-4o-mini
5. Cuaderno del día 2.4: (a) recupera nuestro tasador de Bosque aleatorio (b) crea un tasador de conjunto que permita contribuciones de todos los tasadores

¡Uf! ¡Eso es mucho para hacer en un día!

## TEN EN CUENTA:

Ya tenemos un estimador de productos muy poderoso con nuestro LLM patentado y perfeccionado. ¡La mayoría de las personas estarían muy satisfechas con eso! La razón principal por la que agregamos estos pasos adicionales es para profundizar su experiencia con RAG y con los flujos de trabajo de Agentic.

In [1]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from items import Item
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [2]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
DB = "products_vectorstore"

In [3]:
# Log in en HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Volvamos a los archivos pkl

Aunque disfrutamos mucho de la curación de datos en la semana 6, probablemente no queramos pasar por todo ese proceso nuevamente.

Reutilicemos los archivos pkl que creamos entonces. Copie los archivos `train.pkl` y `test.pkl` de la carpeta de la semana 6 a esta carpeta de la semana 8, o también puede descargarlos desde aquí:

https://drive.google.com/drive/folders/1f_IZGybvs9o0J5sb3xmtTEQB3BXllzrW?usp=drive_link

In [4]:
# Con train.pkl en esta carpeta, puedes ejecutar:

with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

In [5]:
train[0].prompt

'How much does this cost to the nearest dollar?\n\nDelphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7\n\nPrice is $227.00'

# Ahora crea un almacén de datos de Chroma

En la semana 5, creamos un almacén de datos de Chroma con 123 documentos que representan fragmentos de objetos de nuestra empresa ficticia Insurellm.

¡Ahora crearemos un almacén de datos de Chroma con 400 000 productos de nuestro conjunto de datos de entrenamiento! ¡Se está volviendo real!

Ten en cuenta que no usaremos LangChain, pero la API es muy sencilla y coherente con la anterior.

In [6]:
client = chromadb.PersistentClient(path=DB)

In [7]:
# Comprueba si la colección existe y elimínala si es así
collection_name = "products"
existing_collection_names = [collection.name for collection in client.list_collections()]
if collection_name in existing_collection_names:
    client.delete_collection(collection_name)
    print(f"Deleted existing collection: {collection_name}")

collection = client.create_collection(collection_name)

# Presentamos SentenceTransformer

El modelo all-MiniLM es un modelo muy útil de HuggingFace que asigna oraciones y párrafos a un espacio vectorial denso de 384 dimensiones y es ideal para tareas como la búsqueda semántica.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

Puede ejecutarse localmente con bastante rapidez.

La última vez usamos incrustaciones de OpenAI para producir incrustaciones vectoriales. Beneficios en comparación con las incrustaciones de OpenAI:
1. ¡Es gratis y rápido!
2. Podemos ejecutarlo localmente, por lo que los datos nunca salen de nuestra caja; puede ser útil si estás creando un RAG personal

In [8]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Pasa una lista de textos y obtén una matriz numpy de vectores

vector = model.encode(["Hola, ¿cómo estás?"])[0]

In [11]:
vector

array([-1.15370599e-03,  5.10658771e-02,  1.53511772e-02,  2.32141279e-02,
       -5.79879768e-02, -7.84517266e-04,  1.17281683e-01, -1.06943250e-02,
        2.04955712e-02, -1.72842965e-02,  6.00087605e-02, -9.82523989e-03,
       -8.24677423e-02,  2.87922975e-02,  1.30775616e-01,  4.28776480e-02,
       -2.61534397e-02, -2.76041329e-02,  2.44764015e-02, -7.89869577e-03,
        6.07454889e-02,  6.74452493e-03, -7.15552717e-02,  1.07552797e-01,
       -6.41963854e-02, -3.59639712e-02,  4.00140136e-02,  1.86637994e-02,
       -3.14408615e-02, -7.57469609e-02, -3.55393589e-02,  4.59692478e-02,
        2.67851613e-02,  2.39503346e-02,  1.50616253e-02, -1.37406476e-02,
        7.68324211e-02, -1.17426433e-01, -4.72090617e-02,  4.06149738e-02,
       -1.25587568e-01, -1.39257731e-02,  5.64726535e-03,  3.93795734e-03,
        1.87711150e-03, -1.02197789e-01, -4.06028563e-03,  5.26404381e-02,
        5.41950203e-02, -2.73344852e-02, -1.07816517e-01, -9.07926448e-03,
       -6.23978786e-02,  

In [13]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [17]:
description(train[0])

'Delphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7'

In [18]:
for i in tqdm(range(0, len(train), 1000)):
    documents = [description(item) for item in train[i: i+1000]]
    vectors = model.encode(documents).astype(float).tolist()
    metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
    ids = [f"doc_{j}" for j in range(i, i+1000)]
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=vectors,
        metadatas=metadatas
    )

100%|█████████████████████████████████████████| 400/400 [15:49<00:00,  2.37s/it]
